<br>

<center><img src="https://www.htu.edu.jo/images/ThumbnailsCoverPhotos/HTU%20Logo-250px.png" alt="HTU"  width="180px" align="center">


<br>

<p>

**Deep Learning**

10204450

Section (3)

**Developing a deep learning-based system - Interface**

**Submitted to**

Dr. Ala'a Al-Habashna

**Submitted on**

June 13th, 2024

**Submitted by**

Marwan Tarek Shafiq Al Farah

**Student ID**

21110011

Spring 2023 – 2024
</p></center>

# **Importing Libraries**

In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox, Toplevel, Text
from PIL import Image, ImageTk
import torch
import torch.nn as nn
from torchvision import transforms, models, datasets
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

# **Models' Definition**

In [2]:
models_dict = {
    'resnet18': models.resnet18(pretrained=False),
    'vgg16': models.vgg16(pretrained=False),
    'densenet121': models.densenet121(pretrained=False),
    'mobilenet_v2': models.mobilenet_v2(pretrained=False)
}

c:\Users\marwa\miniconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\marwa\miniconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [3]:
num_classes = 14

for model_name, model in models_dict.items():
    if 'resnet' in model_name:
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, num_classes)
    elif 'vgg' in model_name:
        num_ftrs = model.classifier[6].in_features
        model.classifier[6] = nn.Linear(num_ftrs, num_classes)
    elif 'densenet' in model_name:
        num_ftrs = model.classifier.in_features
        model.classifier = nn.Linear(num_ftrs, num_classes)
    elif 'mobilenet' in model_name:
        num_ftrs = model.classifier[-1].in_features
        model.classifier[-1] = nn.Linear(num_ftrs, num_classes)

In [4]:
model_paths = {
    'resnet18': 'D:/Marwan/HTU/Third Year/Spring Semester/Deep Learning/My Solutions/Final/Deep Learning Outputs/best_resnet18.pth',
    'vgg16': 'D:/Marwan/HTU/Third Year/Spring Semester/Deep Learning/My Solutions/Final/Deep Learning Outputs/best_vgg16.pth',
    'densenet121': 'D:/Marwan/HTU/Third Year/Spring Semester/Deep Learning/My Solutions/Final/Deep Learning Outputs/best_densenet121.pth',
    'mobilenet_v2': 'D:/Marwan/HTU/Third Year/Spring Semester/Deep Learning/My Solutions/Final/Deep Learning Outputs/best_mobilenet_v2.pth'
}

In [5]:
for model_name, model_path in model_paths.items():
    models_dict[model_name].load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
    models_dict[model_name].eval()

# **Image Transform and Class Labels**

In [6]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

In [7]:
# Map class indices to class names
class_labels = {
    0: 'cables',
    1: 'case',
    2: 'cpu',
    3: 'gpu',
    4: 'hdd',
    5: 'headset',
    6: 'keyboard',
    7: 'microphone',
    8: 'monitor',
    9: 'motherboard',
    10: 'mouse',
    11: 'ram',
    12: 'speakers',
    13: 'webcam'
}

# **Interface Class Definitions**

## **Initial Interface**

In [8]:
class InitialInterface:
    def __init__(self, root):
        self.root = root
        self.root.title("Initial Interface")

        self.run_test_button = tk.Button(root, text="Run on Entire Testing Set", command=self.run_on_entire_testing_set)
        self.run_test_button.pack(pady=20)

        self.run_selected_button = tk.Button(root, text="Run on Selected Images", command=self.run_on_selected_images)
        self.run_selected_button.pack(pady=20)

    def run_on_entire_testing_set(self):
        self.root.destroy()
        self.open_entire_testing_set_interface()

    def run_on_selected_images(self):
        self.root.destroy()
        self.open_selected_images_interface()

    def open_entire_testing_set_interface(self):
        new_root = tk.Tk()
        new_root.title("Run on Entire Testing Set")
        TestingSetApp(new_root)
        new_root.protocol("WM_DELETE_WINDOW", lambda: self.reopen_initial_interface(new_root))
        new_root.mainloop()

    def open_selected_images_interface(self):
        new_root = tk.Tk()
        app = ImageClassifierApp(new_root)
        new_root.protocol("WM_DELETE_WINDOW", lambda: self.reopen_initial_interface(new_root))
        new_root.mainloop()

    def reopen_initial_interface(self, window):
        window.destroy()
        root = tk.Tk()
        InitialInterface(root)
        root.mainloop()

## **Testing Dataset Interface**

In [9]:
class TestingSetApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Run on Entire Testing Set")

        self.model_name = tk.StringVar(value='resnet18')

        # Dropdown menu to select the model
        self.model_label = tk.Label(root, text="Select Model:")
        self.model_label.pack()
        self.model_menu = tk.OptionMenu(root, self.model_name, *models_dict.keys())
        self.model_menu.pack()

        # Button to run predictions on the entire testing set
        self.run_button = tk.Button(root, text="Run Predictions", command=self.run_predictions)
        self.run_button.pack(pady=20)

        # Buttons for classification report and confusion matrix
        self.report_button = tk.Button(root, text="Show Classification Report", command=self.show_report, state=tk.DISABLED)
        self.report_button.pack(pady=5)
        self.confusion_matrix_button = tk.Button(root, text="Show Confusion Matrix", command=self.show_confusion_matrix, state=tk.DISABLED)
        self.confusion_matrix_button.pack(pady=5)

        # Navigation buttons
        self.prev_button = tk.Button(root, text="Previous", command=self.show_prev_image, state=tk.DISABLED)
        self.prev_button.pack(side=tk.LEFT)
        self.next_button = tk.Button(root, text="Next", command=self.show_next_image, state=tk.DISABLED)
        self.next_button.pack(side=tk.RIGHT)

        # Label to show the selected image
        self.image_label = tk.Label(root)
        self.image_label.pack()

        # Labels to show prediction results
        self.correct_label = tk.Label(root, text="")
        self.correct_label.pack()
        self.actual_class_label = tk.Label(root, text="")
        self.actual_class_label.pack()
        self.predicted_class_label = tk.Label(root, text="")
        self.predicted_class_label.pack()

        # Variables to store images and predictions
        self.images = []
        self.predictions = []
        self.actual_classes = []
        self.current_image_index = 0

        # Load the dataset
        self.dataset = datasets.ImageFolder(root='D:/Marwan/HTU/Third Year/Spring Semester/Deep Learning/My Solutions/Final/pc_parts', transform=transform)
        train_size = int(0.7 * len(self.dataset))
        val_size = int(0.15 * len(self.dataset))
        test_size = len(self.dataset) - train_size - val_size
        torch.manual_seed(42)
        _, _, self.dataset = random_split(self.dataset, [train_size, val_size, test_size])
        self.dataloader = DataLoader(self.dataset, batch_size=1, shuffle=False)

    def run_predictions(self):
        self.images = []
        self.predictions = []
        self.actual_classes = []
        self.current_image_index = 0

        model = models_dict[self.model_name.get()]

        for inputs, labels in self.dataloader:
            image = inputs.squeeze(0)
            self.images.append(image)
            actual_class = class_labels[labels.item()]
            self.actual_classes.append(actual_class)

            with torch.no_grad():
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                predicted_class = class_labels[preds.item()]
                self.predictions.append(predicted_class)

        self.show_image()
        self.update_buttons()
        self.report_button.config(state=tk.NORMAL)
        self.confusion_matrix_button.config(state=tk.NORMAL)

    def show_image(self):
        if not self.images:
            return
        image = self.images[self.current_image_index]
        image = transforms.ToPILImage()(image)
        image.thumbnail((200, 200))
        self.image = ImageTk.PhotoImage(image)
        self.image_label.config(image=self.image)

        if self.predictions[self.current_image_index] == self.actual_classes[self.current_image_index]:
            self.correct_label.config(text="Correctly Classified")
        else:
            self.correct_label.config(text="Misclassified")

        self.actual_class_label.config(text=f"Actual Class: {self.actual_classes[self.current_image_index]}")
        self.predicted_class_label.config(text=f"Predicted Class: {self.predictions[self.current_image_index]}")
        self.update_buttons()

    def update_buttons(self):
        if self.current_image_index == 0:
            self.prev_button.config(state=tk.DISABLED)
        else:
            self.prev_button.config(state=tk.NORMAL)
        if self.current_image_index == len(self.images) - 1:
            self.next_button.config(state=tk.DISABLED)
        else:
            self.next_button.config(state=tk.NORMAL)

    def show_prev_image(self):
        if self.current_image_index > 0:
            self.current_image_index -= 1
            self.show_image()

    def show_next_image(self):
        if self.current_image_index < len(self.images) - 1:
            self.current_image_index += 1
            self.show_image()

    def show_report(self):
        report = classification_report(self.actual_classes, self.predictions, target_names=[class_labels[i] for i in range(num_classes)])
        report_window = Toplevel(self.root)
        report_window.title("Classification Report")
        report_text = Text(report_window, wrap=tk.WORD)
        report_text.pack(expand=True, fill=tk.BOTH)
        report_text.insert(tk.END, report)

    def show_confusion_matrix(self):
        cm = confusion_matrix(self.actual_classes, self.predictions)
        cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cm_window = Toplevel(self.root)
        cm_window.title("Confusion Matrix")

        fig, ax = plt.subplots(figsize=(10, 8))
        sns.heatmap(cm_normalized, annot=True, cmap="Blues", xticklabels=[class_labels[i] for i in range(num_classes)], yticklabels=[class_labels[i] for i in range(num_classes)], ax=ax)
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.title('Confusion Matrix')

        canvas = FigureCanvasTkAgg(fig, master=cm_window)
        canvas.draw()
        canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)

## **Image Classifier**

In [10]:
class ImageClassifierApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Classifier")

        self.model_name = tk.StringVar(value='resnet18')

        # Dropdown menu to select the model
        self.model_label = tk.Label(root, text="Select Model:")
        self.model_label.pack()
        self.model_menu = tk.OptionMenu(root, self.model_name, *models_dict.keys())
        self.model_menu.pack()

        # Button to browse image
        self.browse_button = tk.Button(root, text="Browse Images", command=self.browse_images)
        self.browse_button.pack()

        # Label to show the selected image
        self.image_label = tk.Label(root)
        self.image_label.pack()

        # Button to predict
        self.predict_button = tk.Button(root, text="Predict", command=self.predict)
        self.predict_button.pack()

        # Navigation buttons
        self.prev_button = tk.Button(root, text="Previous", command=self.show_prev_image, state=tk.DISABLED)
        self.prev_button.pack(side=tk.LEFT)
        self.next_button = tk.Button(root, text="Next", command=self.show_next_image, state=tk.DISABLED)
        self.next_button.pack(side=tk.RIGHT)

        # Label to show the prediction result
        self.result_label = tk.Label(root, text="")
        self.result_label.pack()

        # Variables to store images and predictions
        self.images = []
        self.image_paths = []
        self.current_image_index = 0

    def browse_images(self):
        self.filepaths = filedialog.askopenfilenames(filetypes=[("Image files", "*.jpg *.jpeg *.png")])
        if not self.filepaths:
            return
        self.images = [Image.open(filepath) for filepath in self.filepaths]
        self.image_paths = self.filepaths
        self.current_image_index = 0
        self.show_image()

    def show_image(self):
        image = self.images[self.current_image_index]
        image.thumbnail((200, 200))
        self.image = ImageTk.PhotoImage(image)
        self.image_label.config(image=self.image)
        self.result_label.config(text="")
        self.update_buttons()

    def update_buttons(self):
        if self.current_image_index == 0:
            self.prev_button.config(state=tk.DISABLED)
        else:
            self.prev_button.config(state=tk.NORMAL)
        if self.current_image_index == len(self.images) - 1:
            self.next_button.config(state=tk.DISABLED)
        else:
            self.next_button.config(state=tk.NORMAL)

    def show_prev_image(self):
        if self.current_image_index > 0:
            self.current_image_index -= 1
            self.show_image()
            self.result_label.config(text=f"Predicted Class: {self.predictions[self.current_image_index]}")

    def show_next_image(self):
        if self.current_image_index < len(self.images) - 1:
            self.current_image_index += 1
            self.show_image()
            self.result_label.config(text=f"Predicted Class: {self.predictions[self.current_image_index]}")

    def predict(self):
        if not self.filepaths:
            messagebox.showerror("Error", "Please select images first.")
            return

        model = models_dict[self.model_name.get()]
        self.predictions = []

        for filepath in self.filepaths:
            image = Image.open(filepath).convert('RGB')
            image = transform(image)
            image = image.unsqueeze(0)

            with torch.no_grad():
                outputs = model(image)
                _, preds = torch.max(outputs, 1)
                predicted_class = class_labels[preds.item()]
                self.predictions.append(predicted_class)

        self.show_image()
        self.result_label.config(text=f"Predicted Class: {self.predictions[self.current_image_index]}")

# **Main Code**

In [11]:
root = tk.Tk()
InitialInterface(root)
root.mainloop()